# 🍎 Apple Detection - Google Colab Setup

This notebook will help you set up and run the Apple Detection project on Google Colab.

## Steps:
1. Enable GPU runtime
2. Clone/upload the project
3. Install dependencies
4. Upload your dataset
5. Train the model
6. Run inference


## Step 1: Enable GPU Runtime

**Important:** Before running any cells, enable GPU:
1. Go to **Runtime** → **Change runtime type**
2. Set **Hardware accelerator** to **GPU** (T4)
3. Click **Save**

Let's verify GPU is available:


In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  GPU not available. Please enable GPU runtime.")
    print("   Go to Runtime → Change runtime type → GPU")


## Step 2: Clone or Upload Project

### Option A: Clone from GitHub (if you have a repository)


In [ ]:
# Uncomment and modify if you have a GitHub repository
# !git clone https://github.com/yourusername/apple-detection.git
# %cd apple-detection


### Option B: Upload Project Files

If you don't have a GitHub repo, you can upload your project files:
1. Create a zip file of your project (excluding `venv/`, `__pycache__/`, etc.)
2. Upload it using the cell below
3. Extract it


In [ ]:
from google.colab import files
import zipfile
import os

# Upload zip file
uploaded = files.upload()

# Extract if zip file was uploaded
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content')
        print(f"✅ Extracted to /content/")
        # Remove zip file
        os.remove(filename)

# Navigate to project directory (adjust name if different)
# %cd /content/apple-detection


## Step 3: Install Dependencies

Install all required packages for the project:


In [ ]:
# Install PyTorch with CUDA support (Colab usually has this, but we'll ensure it)
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies
%pip install -r /content/apple-detection/requirements.txt

# Verify installation
import torch
import torchvision
import numpy as np
import cv2
import yaml

print("✅ All dependencies installed successfully!")
print(f"   PyTorch: {torch.__version__}")
print(f"   Torchvision: {torchvision.__version__}")
print(f"   NumPy: {np.__version__}")


## Step 4: Upload Dataset

Upload your dataset. You have several options:

### Option A: Upload from Local Computer
Upload a zip file containing your dataset:


In [ ]:
from google.colab import files
import zipfile
import os

# Create data directory structure
os.makedirs('/content/apple-detection/data/images/train', exist_ok=True)
os.makedirs('/content/apple-detection/data/images/val', exist_ok=True)
os.makedirs('/content/apple-detection/data/images/test', exist_ok=True)
os.makedirs('/content/apple-detection/data/annotations/train', exist_ok=True)
os.makedirs('/content/apple-detection/data/annotations/val', exist_ok=True)
os.makedirs('/content/apple-detection/data/annotations/test', exist_ok=True)

# Upload dataset zip file
uploaded = files.upload()

# Extract dataset
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('/content/apple-detection/data')
        print(f"✅ Dataset extracted!")
        os.remove(filename)


### Option B: Mount Google Drive

If your dataset is in Google Drive:


In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Copy dataset from Drive to Colab (adjust path as needed)
# !cp -r /content/drive/MyDrive/your-dataset-path/* /content/apple-detection/data/


### Verify Dataset Structure

Check that your dataset is properly organized:


In [ ]:
import os
from pathlib import Path

data_dir = Path('/content/apple-detection/data')

print("📁 Dataset Structure:")
print(f"   Root: {data_dir}")

# Check images
for split in ['train', 'val', 'test']:
    img_dir = data_dir / 'images' / split
    ann_dir = data_dir / 'annotations' / split
    
    img_count = len(list(img_dir.glob('*.jpg')) + list(img_dir.glob('*.png'))) if img_dir.exists() else 0
    ann_count = len(list(ann_dir.glob('*.txt'))) if ann_dir.exists() else 0
    
    print(f"\n   {split.upper()}:")
    print(f"      Images: {img_count}")
    print(f"      Annotations: {ann_count}")

print("\n✅ Dataset structure verified!")


## Step 5: Train the Model

Now you're ready to train! Use the Colab-optimized configuration:


In [ ]:
# Add src to path
import sys
sys.path.insert(0, '/content/apple-detection/src')

# Train the model
# !python /content/apple-detection/src/train.py --config /content/apple-detection/configs/config_colab.yaml

# Or if you prefer to run in Python:
# from src.train import train_model
# train_model(config_path='/content/apple-detection/configs/config_colab.yaml')


## Step 6: Run Inference

Test your trained model on new images:


In [ ]:
# Upload a test image
from google.colab import files
from IPython.display import Image, display

uploaded = files.upload()

# Run inference (adjust paths as needed)
# !python /content/apple-detection/src/inference.py \
#     --image /content/your_test_image.jpg \
#     --checkpoint /content/apple-detection/checkpoints/best_model.pth \
#     --output /content/apple-detection/results/detection.jpg \
#     --config /content/apple-detection/configs/config_colab.yaml

# Display result
# display(Image('/content/apple-detection/results/detection.jpg'))


## Step 7: Save Your Work

**Important:** Colab sessions are temporary. Save your checkpoints and results to Google Drive:


In [ ]:
# Mount drive if not already mounted
from google.colab import drive
drive.mount('/content/drive')

# Create backup directory in Drive
import shutil
backup_dir = '/content/drive/MyDrive/apple-detection-backup'
os.makedirs(backup_dir, exist_ok=True)

# Copy checkpoints
if os.path.exists('/content/apple-detection/checkpoints'):
    shutil.copytree('/content/apple-detection/checkpoints', 
                    f'{backup_dir}/checkpoints', 
                    dirs_exist_ok=True)
    print("✅ Checkpoints saved to Drive!")

# Copy results
if os.path.exists('/content/apple-detection/results'):
    shutil.copytree('/content/apple-detection/results', 
                    f'{backup_dir}/results', 
                    dirs_exist_ok=True)
    print("✅ Results saved to Drive!")


## 💡 Tips for Colab

1. **Session Timeout**: Colab sessions disconnect after ~90 minutes of inactivity. Keep the tab active during training.

2. **GPU Limits**: Free Colab has usage limits. If you hit them, wait a few hours or consider Colab Pro.

3. **Save Frequently**: Always save checkpoints to Google Drive to avoid losing progress.

4. **Large Datasets**: For large datasets, consider using Google Drive instead of uploading directly.

5. **Monitor Training**: Use TensorBoard or print statements to monitor training progress.

## 🎉 You're All Set!

Happy training! 🍎🔍
